# Interstitial Formation Energy of silicon

<b>Task</b>: to find the interstitial formation energy in bulk silicon.

Self-interstitial atoms are atoms of the same matrix material occupying positions which do not coincide with lattice points.[1] The silicon self-interstitial possesses a <110\> or <100\> dumbbell configuration [2], i.e. 2 atoms  share the same lattice site, and they are displaced symmetrically from it along either $[1, 1, 0]$ or $[1, 0, 0]$

The interstitial formation energy can be found by the equation: <center>$E_i^f = E_f - N_i \cdot E_{ref}$ </center> where $E_f$, $N_i$ and $E_{ref}$ are the final energy of the relaxed system with interstitial, the number of atoms in the system with interstitial and energy per unit volume of the material respectively.

The energy of the cluster is calculated using the Stillger-Weber potential, which is implemented in `JuLIP` as StillingerWeber().

A table showing how the interstitial formation energies change with cluster size is generated at the end.

<b>References</b>:

[1]https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_Interstitial_Formation_Energy

[2]Frank, W., Seeger, A., & Gösele, U. (1980). From the Mystery to the Understanding of the Self-Interstitials in Silicon. MRS Proceedings, 2, 31. doi:10.1557/PROC-2-31

In [1]:
using JuLIP, Optim, DataFrames

In [2]:
# reference energy
# energy per unit volume of a homogeneous silicon crystal
at = bulk(:Si)
set_calculator!(at, StillingerWeber())
Eref = energy(at) / length(at)

-4.33499825548334

In [3]:
function IFE(size, config)
    # atuster with interstitial
    at = bulk(:Si, cubic = true) * size
    set_calculator!(at, StillingerWeber())

    disp = 0.5 * rnn(:Si)
    i = length(at) ÷ 2
    site = at.X[i]
    deleteat!(at, i)
    insert!(at.X, i, site + config * disp)
    insert!(at.X, i, site - config * disp)

    # energy before relaxing
    E0 = energy(at) - length(at) * Eref
    println(length(at))

    # geometry optimization
    set_constraint!(at, FixedCell(at))
    result = minimise!(at)
    E1 = result.minimum - length(at) * Eref

    return E0, E1
end

IFE (generic function with 1 method)

In [ ]:
# # defect formation energy
size_range = 3:10
IFE_110_0, IFE_110_1 = [ IFE(n, [1, 1, 0])[1] for n in size_range ], [ IFE(n, [1, 1, 0])[2] for n in size_range ]
IFE_100_0, IFE_100_1 = [ IFE(n, [1, 0, 0])[1] for n in size_range ], [ IFE(n, [1, 0, 0])[2] for n in size_range ]

In [5]:
df = DataFrame(Size = size_range, Unrelaxed_IFE_110 = IFE_110_0, Relaxed_IFE_110 = IFE_110_1, 
        Unrelaxed_IFE_100 = IFE_100_0, Relaxed_IFE_100 = IFE_100_1)
display(df)


,Size,Unrelaxed_IFE_110,Relaxed_IFE_110,Unrelaxed_IFE_100,Relaxed_IFE_100
1,3,44.0452,4.47726,13.2659,5.31058
2,4,44.0452,4.44509,13.2659,5.28864
3,5,44.0452,4.43438,13.2659,5.2814
4,6,44.0452,4.42979,13.2659,5.27829
5,7,44.0452,4.42748,13.2659,5.27672
6,8,44.0452,4.42619,13.2659,5.27584
7,9,44.0452,4.42542,13.2659,5.27531
8,10,44.0452,4.42492,13.2659,5.27497
